# Keeper League Workflow



In [64]:
import pandas as pd
import os
from functions.my_leagues import *
import numpy as np

In [65]:
league = '792216196090019840'

In [66]:
players = pd.read_csv('Files/players.csv', dtype='object')

In [67]:
rosters = get_league_rosters(league_id=league)#key='ons')

In [68]:
teams = pd.DataFrame()
for index,row in rosters.iterrows():
    owner = row['owner_id']
    owner_list = [owner for p in row['players']]
    df = pd.DataFrame(owner_list, row['players']).reset_index()
    df.columns = ['player_id','owner_id']
    teams = teams.append(df)

print(teams.head())
             

  player_id            owner_id
0      1049  335516722092052480
1      1234  335516722092052480
2      1837  335516722092052480
3      2319  335516722092052480
4      2410  335516722092052480


In [69]:
league = get_league(league_id=league)#key='ons')

In [121]:
vor = pd.read_csv('Files/vor.csv', dtype='object')
vor = vor[vor['scoring_type'] == 'dynasty_2qb']
vor = vor[vor['league_size'] == '12']
vor['adp'] = vor['adp'].astype(float)
vor['vor'] = vor['vor'].astype(float)

In [122]:
league_data = league[['user_id','display_name']].merge(teams, left_on='user_id',right_on='owner_id',how='left').drop('owner_id',axis=1)
dataset = league_data.merge(players[['player_id','full_name','position','team']], on='player_id',how='left')
dataset = dataset.merge(vor[['player_id','adp','vor']], on='player_id',how='left')
#dataset = dataset.merge(draft_results[['player_id','round']], on='player_id',how='left')
dataset.to_csv('Files/dynasty_rosters.csv',index=False)

In [123]:
vor.describe()

,adp,vor
count,1120.000000,599.000000
mean,673.430804,-51.798331
std,370.384181,83.457800
min,1.700000,-297.740000
25%,268.525000,-100.500000
50%,999.000000,-36.500000
75%,999.000000,-3.000000
max,999.000000,185.400000


In [124]:
my_roster = dataset[dataset['display_name'] == 'windleypratt'].sort_values(by=['vor','adp'], ascending = [False,True])

In [125]:
my_roster

,user_id,display_name,player_id,full_name,position,team,adp,vor
7,335446335220121600,windleypratt,4034,Christian McCaffrey,RB,CAR,2.1,168.30
16,335446335220121600,windleypratt,6794,Justin Jefferson,WR,MIN,5.9,151.92
1,335446335220121600,windleypratt,2449,Stefon Diggs,WR,BUF,11.2,113.40
8,335446335220121600,windleypratt,4137,James Conner,RB,ARI,31.4,100.60
14,335446335220121600,windleypratt,5927,Terry McLaurin,WR,WAS,37.7,66.80
17,335446335220121600,windleypratt,6904,Jalen Hurts,QB,PHI,65.2,45.56
10,335446335220121600,windleypratt,5001,Dalton Schultz,TE,DAL,56.4,24.90
13,335446335220121600,windleypratt,5916,Darrell Henderson,RB,LAR,145.7,20.40
22,335446335220121600,windleypratt,8135,Treylon Burks,WR,TEN,95.1,2.20
11,335446335220121600,windleypratt,5086,Marquez Valdes-Scantling,WR,KC,137.9,-1.40


In [126]:
available = players[['player_id','full_name','position','team', 'years_exp']].merge(vor[['player_id','adp','vor']], on='player_id',how='left').dropna(subset=['adp'])
available = available.merge(league_data, on='player_id', how='left')
available = available[~available['position'].isin(['DEF','K'])]
available = available[(available['display_name'] == 'windleypratt') | (available['display_name'].isnull())]

available.sort_values(by=['adp','vor'], ascending=[True,False]).to_csv('Files/available_players.csv', index=False)


In [127]:
dataset = dataset.sort_values(by=['user_id', 'vor'], ascending=[True,False])
ratings = dataset.groupby(['display_name', 'position']).head(3).sort_values(by=['position','user_id', 'vor'], ascending=[True, True, False]).to_csv('Files/team_ratings_data.csv', index=False)

In [128]:
limits = {'QB': 2, 'WR': 3, 'RB': 2, 'TE': 1}

In [129]:
ratings = pd.DataFrame()
for x in limits:
    n = limits.get(x)
    dfR = dataset[dataset['position'] == x].groupby(['display_name', 'position']).head(n)
    ratings = ratings.append(dfR)

In [130]:
ratings

,user_id,display_name,player_id,full_name,position,team,adp,vor
17,335446335220121600,windleypratt,6904,Jalen Hurts,QB,PHI,65.2,45.56
4,335446335220121600,windleypratt,3161,Carson Wentz,QB,WAS,255.6,-60.18
27,335516722092052480,itsmo,1234,Russell Wilson,QB,DEN,73.5,18.94
26,335516722092052480,itsmo,1049,Ryan Tannehill,QB,TEN,213.6,-43.84
59,340205118542610432,rtpleasa,4984,Josh Allen,QB,BUF,22.3,70.46
...,...,...,...,...,...,...,...,...
192,342852494541471744,Pangle,7600,Pat Freiermuth,TE,PIT,104.5,15.90
202,725887513603932160,lakedawson,2505,Darren Waller,TE,LV,40.9,54.20
229,792484068049686528,yejinkim4,5012,Mark Andrews,TE,BAL,23.6,77.00
251,792484337131032576,abnw24,1466,Travis Kelce,TE,KC,13.2,103.70


In [131]:
ratings.groupby(['display_name','position'])['vor'].sum().reset_index().sort_values(by=['position', 'vor'], ascending=[True, False])

,display_name,position,vor
32,ndkauffm,QB,62.92
12,abnw24,QB,52.38
4,Mbalexan,QB,43.78
16,alexlane89,QB,39.28
36,rtpleasa,QB,20.24
24,jspardue,QB,16.92
0,Jprice08,QB,12.90
44,yejinkim4,QB,0.04
28,lakedawson,QB,-12.76
40,windleypratt,QB,-14.62


,user_id,display_name,player_id
0,335446335220121600,windleypratt,2431
1,335446335220121600,windleypratt,2449
2,335446335220121600,windleypratt,2711
3,335446335220121600,windleypratt,2749
4,335446335220121600,windleypratt,3161
...,...,...,...
297,792485519014625280,Jprice08,8130
298,792485519014625280,Jprice08,8138
299,792485519014625280,Jprice08,8140
300,792485519014625280,Jprice08,8174
